# EMS Data Analysis

In [4]:
import os.path
import pandas as pd
from bokeh.core.properties import value
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.io import export_png

output_notebook()

def plot_incidents(data, name):
    year_names = [str(x) for x in data.index.values]
    years =  year_names
    y_axis = ["geom", "no-geo", "bi"]
    colors = ["#5CC9F5", "#e84d60", "#cccccc"]
    
    # Count number without geometry
    data['no-geo'] = data['count'] - data['geom']
    data['bi'] = data['count'] - data['bi_count']


    parsed = {
        'years': years,
        'geom': data['geom'],
        'no-geo': data['no-geo'],
        'bi': data['bi']
    }

    p = figure(x_range=years, plot_height=250, title=name,
               toolbar_location=None)

    p.vbar_stack(y_axis, x='years', width=0.9, color=colors, source=parsed,
                 legend=[value(x) for x in y_axis])

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"

    show(p)
#     export_png(p, filename="figures/%s.png" % (name,))

def plot_state_incidents(name):
    data = states[name]
    plot_incidents(data, name)
    


Loading BokehJS ...

## Incidents by year

# SQL Queries

`
select count(1), sum(case when exp_no = 0 then 1 else 0 end), extract(year from inc_date) as year 
from ems.ems group by year;
`

`
SELECT count(1), extract(year from alarm) as year
FROM ems.basicincident a
WHERE a.state = 'DC'
GROUP by year
`

## Incidents by state

In [6]:
# TOTALS
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
         'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
         'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
         'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
         'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
         'DC']

states = {} # Stores the updated query data
basicincident = {} # Basic incident totals by year
# Load all states in the `codes` list
for code in state_codes:
    filename = 'data/updated/%s.csv' % (code,)
    basicincident_filename = 'data/basicincident/%s.csv' % (code,)
    if os.path.isfile(filename):
        # Load into a dict with the state as key
        states[code] = pd.read_csv(filename)
        basicincident = pd.read_csv(basicincident_filename)
        
        # Merge
        states[code]['bi_count'] = basicincident['count']
        # Set the year as index
        states[code].set_index('year', inplace=True)

    else:
        print("Did not load: %s" % (code,))
        
for code in state_codes:
    plot_state_incidents(code)

In [3]:
total = 0
with_geom = 0
no_geom = 0
bi_count = 0

# Calcultate joined entrie totals
for state in states.values():
    total += sum(state['count'])
    with_geom += sum(state['geom'])
    no_geom += sum(state['no-geo'])
    bi_count += sum(state['bi_count'])
    
    
print("Total %s" % (total,))
print("With %s" % (with_geom,))
print("Without %s" % (no_geom,))
print("Ratio: %s" % (with_geom / total,))  
print("BasicIncidents: %s" % (bi_count))
print("Ratio of entries Updated: %s" % (bi_count/total))

# Chart
x_axis = ["Entries",]
y_axis = ["geom", "no-geo", "bi"]
colors = ["#5CC9F5", "#e84d60", "#cccccc"]
parsed = { 'total': x_axis,
           'geom': [with_geom],
           'no-geo': [no_geom],
           'bi': [(total-bi_count)]}

p = figure(x_range=x_axis, plot_height=250, title="Total Entries",
           toolbar_location=None)

p.vbar_stack(y_axis, x='total', width=0.9, color=colors, source=parsed,
             legend=[value(x) for x in y_axis])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)



Total 117383071
With 75929729
Without 41453342
Ratio: 0.6468541703087662
BasicIncidents: 117333294
Ratio of entries Updated: 0.9995759439621409


In [7]:
# Calculate other table totals
total_ems = 0
total_basic = 0
total_address = 0

ems = pd.read_csv('data/ems_totals_by_state.csv')
total_ems = sum(ems['count'])
print("Total EMS entries: %s" % (total_ems,))

basic = pd.read_csv('data/basicincident_totals_by_state.csv')
total_basic = sum(basic['count'])
print("Total basicaddress entries: %s" % (total_basic,))

address = pd.read_csv('data/incidentaddress_total_by_state.csv')
total_address = sum(address['count'])
print("Total incidentaddress entries: %s" % (total_address,))

Total EMS entries: 8131224
Total basicaddress entries: 117353939
Total incidentaddress entries: 90015396


In [8]:
# for state in states:
#     for year in state.years:
allStates = pd.DataFrame()
joined = pd.DataFrame()

for state in states.values():
    allStates = allStates.add(state, fill_value=0)
        
allStates.head()

joined = pd.concat(states)
joined.drop(columns=['bi', 'bi_count'])
    
# plot_state_incidents(allStates)


count     geom   no-geo
   year                           
AK 2012    58644    58597       47
   2013    61502    61501        1
   2014    60638        0    60638
   2015    64571    64571        0
   2016    67415        0    67415
AL 2012   249674        0   249674
   2013   209036        0   209036
   2014   260470        0   260470
   2015   322581   322579        2
   2016   344077        0   344077
AR 2012   188622   160542    28080
   2013   187020   158853    28167
   2014   206000    27108   178892
   2015   199764   199760        4
   2016   213748        0   213748
AZ 2012   358123    87963   270160
   2013   433894   138399   295495
   2014   453354   453354        0
   2015   509052   509051        1
   2016   538248        0   538248
CA 2012  1404589  1292220   112369
   2013  2126138  2022739   103399
   2014  2162715  2162710        5
   2015  2275302  2275298        4
   2016  2589083        0  2589083
CO 2012   460093    37267   422826
   2013   490225    52778   437447
   2014   520307   520306        1
   2015   560928   560927        1
   2016   615939        0   615939
...          ...      ...      ...
VA 2012   785633   505710   279923
   2013   796308   512415   283893
   2014   827142   827136        6
   2015   851647   851643        4
   2016   884513        0   884513
VT 2012    43953    21016    22937
   2013    45012    21071    23941
   2014    45410    45410        0
   2015    46207    46207        0
   2016    44056        0    44056
WA 2012   548023   548011       12
   2013   519765   247792   271973
   2014   569696   569695        1
   2015   633483   633482        1
   2016   559599        0   559599
WI 2012   179111   161767    17344
   2013   211300        0   211300
   2014   248363   248362        1
   2015   285213   285211        2
   2016   305158        0   305158
WV 2012   127742   108988    18754
   2013   119948        0   119948
   2014   128265   128263        2
   2015   128424   128419        5
   2016   131319        0   131319
WY 2012    30937    24277     6660
   2013    31320        0    31320
   2014    25245    25245        0
   2015    32333    32333        0
   2016    26267        0    26267

[253 rows x 3 columns]

In [9]:
# joined.sum()
allStates.loc[2012]['no-geo']

7307606.0

In [10]:
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import Colorblind
from bokeh.plotting import figure
from bokeh.transform import cumsum

geom = allStates.loc[2012]['geom']
non = allStates.loc[2012]['no-geo']


x = {
    '2012': allStates.loc[2012]['no-geo'],
    '2013': allStates.loc[2013]['no-geo'],
    '2014': allStates.loc[2014]['no-geo'],
    '2015': allStates.loc[2015]['no-geo'],
    '2016': allStates.loc[2016]['no-geo'],
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Colorblind[len(x)]

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
show(p)
    

In [11]:
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import Colorblind
from bokeh.plotting import figure
from bokeh.transform import cumsum

geom = allStates.loc[2012]['geom']
non = allStates.loc[2012]['no-geo']

x = {
    'Geom': geom,
    'No Geom': non,
    'Not updated': 0
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Colorblind[len(x)]

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
show(p)

In [12]:
joined.sum()

count       117383071
geom         75929729
bi_count    117333294
no-geo       41453342
bi              49777
dtype: int64

In [13]:
percentage_without_geom = joined.sum()['no-geo'] * 100 / joined.sum()['count']
print("without geom %%%s" % (percentage_without_geom,))

without geom %35.31458296912338


In [14]:
plot_incidents(allStates, "Totals")

In [15]:
# Calculate other table totals
total_ems = 0
total_basic = 0
total_address = 0

ems = pd.read_csv('data/ems_totals_by_state.csv')
total_ems = sum(ems['count'])
print("Total EMS entries: %s" % (total_ems,))

basic = pd.read_csv('data/basicincident_totals_by_state.csv')
total_basic = sum(basic['count'])
print("Total basicaddress entries: %s" % (total_basic,))

address = pd.read_csv('data/incidentaddress_total_by_state.csv')
total_address = sum(address['count'])
print("Total incidentaddress entries: %s" % (total_address,))

Total EMS entries: 8131224
Total basicaddress entries: 117353939
Total incidentaddress entries: 90015396
